In [55]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

if x_train.shape != (60000, 28, 28, 1):
    print("Đang reshape lại x_train...")
    x_train = x_train.reshape(60000, 28, 28, 1)
    x_test = x_test.reshape(10000, 28, 28, 1)

datagen = ImageDataGenerator(
    rotation_range=15,       # Xoay ảnh ngẫu nhiên +/- 15 độ
    width_shift_range=0.1,   # Dịch chuyển chiều rộng 10%
    height_shift_range=0.1,  # Dịch chuyển chiều cao 10%
    zoom_range=0.1,          # Phóng to/thu nhỏ ngẫu nhiên 10%
    shear_range=0.1          # Làm nghiêng ảnh (shear)
)

datagen.fit(x_train)

print("ImageDataGenerator đã sẵn sàng.")

ImageDataGenerator đã sẵn sàng.


In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = keras.Sequential()

# ---- KHỐI 1 ----
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25)) # Tắt 25% nơ-ron

# ---- KHỐI 2 ----
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25)) # Tắt 25% nơ-ron

# ---- KHỐI 3 (PHÂN LOẠI) ----
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5)) # Tắt 50% nơ-ron
model.add(Dense(num_classes, activation='softmax')) 

print("Model 2.0 (với Dropout) đã được xây dựng.")
model.summary()

Model 2.0 (với Dropout) đã được xây dựng.


/Users/minhtuansfile/HoangTuan_Code/XLA_BTL/venv/lib/python3.13/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,034 (879.04 KB)

 Trainable params: 225,034 (879.04 KB)

 Non-trainable params: 0 (0.00 B)

In [57]:
# ---------------
# BƯỚC 3 (MỚI): BIÊN DỊCH MODEL
# ---------------

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("Mô hình đã được biên dịch.")

Mô hình đã được biên dịch.


In [58]:
num_epochs = 30 # Tăng lên 30 (nhưng EarlyStopping sẽ dừng hộ chúng ta)
batch_size = 64 # Giữ nguyên batch_size 64

# Callbacks (Giữ nguyên từ Step 6 cũ)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=10, # Dừng nếu val_loss không cải thiện sau 10 epoch
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        verbose=1
    ),
    ModelCheckpoint(
        filepath="best_model_augmented.h5", # Đổi tên file để không nhầm lẫn
        monitor="val_accuracy",
        save_best_only=True,
        verbose=1
    )
]

print("Bắt đầu huấn luyện với Data Augmentation...")

# Đây là lệnh fit MỚI
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=batch_size), # DÙNG DATAGEN.FLOW
    epochs=num_epochs,
    validation_data=(x_test, y_test), # Dữ liệu test vẫn giữ nguyên
    callbacks=callbacks,
    verbose=1
)

print("\n...Huấn luyện hoàn tất!")
print("\n🎉 Model TỔNG QUÁT HÓA tốt nhất được lưu tại file: best_model_augmented.h5")

Bắt đầu huấn luyện với Data Augmentation...
Epoch 1/30
931/938 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6661 - loss: 0.9902
Epoch 1: val_accuracy improved from None to 0.97830, saving model to best_model_augmented.h5


938/938 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.8096 - loss: 0.5916 - val_accuracy: 0.9783 - val_loss: 0.0609 - learning_rate: 0.0010
Epoch 2/30
930/938 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9180 - loss: 0.2698
Epoch 2: val_accuracy improved from 0.97830 to 0.98730, saving model to best_model_augmented.h5


938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9233 - loss: 0.2509 - val_accuracy: 0.9873 - val_loss: 0.0381 - learning_rate: 0.0010
Epoch 3/30
931/938 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9402 - loss: 0.1975
Epoch 3: val_accuracy improved from 0.98730 to 0.98960, saving model to best_model_augmented.h5


938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9428 - loss: 0.1888 - val_accuracy: 0.9896 - val_loss: 0.0283 - learning_rate: 0.0010
Epoch 4/30
937/938 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9504 - loss: 0.1688
Epoch 4: val_accuracy improved from 0.98960 to 0.99010, saving model to best_model_augmented.h5


938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9520 - loss: 0.1628 - val_accuracy: 0.9901 - val_loss: 0.0283 - learning_rate: 0.0010
Epoch 5/30
932/938 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9565 - loss: 0.1444
Epoch 5: val_accuracy improved from 0.99010 to 0.99210, saving model to best_model_augmented.h5


...

938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9637 - loss: 0.1208 - val_accuracy: 0.9938 - val_loss: 0.0183 - learning_rate: 0.0010
Epoch 8/30
936/938 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9657 - loss: 0.1159
Epoch 8: val_accuracy did not improve from 0.99380
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9661 - loss: 0.1157 - val_accuracy: 0.9925 - val_loss: 0.0221 - learning_rate: 0.0010
Epoch 9/30
932/938 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9673 - loss: 0.1079
Epoch 9: val_accuracy did not improve from 0.99380
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9671 - loss: 0.1085 - val_accuracy: 0.9937 - val_loss: 0.0182 - learning_rate: 0.0010
Epoch 10/30
937/938 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9693 - loss: 0.1004
Epoch 10: val_accuracy did not improve from 0.99380
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9693 - loss: 0.1005 - val_accuracy: 0.9935 - val_loss: 0.0187 - learning_rate: 0.0010
Epoch 11/30
931/938 ━━━

938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9702 - loss: 0.0992 - val_accuracy: 0.9944 - val_loss: 0.0179 - learning_rate: 0.0010
Epoch 12/30
932/938 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9708 - loss: 0.0968
Epoch 12: val_accuracy did not improve from 0.99440
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9709 - loss: 0.0971 - val_accuracy: 0.9937 - val_loss: 0.0177 - learning_rate: 0.0010
Epoch 13/30
936/938 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9721 - loss: 0.0918
Epoch 13: val_accuracy did not improve from 0.99440
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9730 - loss: 0.0911 - val_accuracy: 0.9936 - val_loss: 0.0196 - learning_rate: 0.0010
Epoch 14/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9742 - loss: 0.0888
Epoch 14: val_accuracy improved from 0.99440 to 0.99500, saving model to best_model_augmented.h5


938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9734 - loss: 0.0905 - val_accuracy: 0.9950 - val_loss: 0.0160 - learning_rate: 0.0010
Epoch 15/30
930/938 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9729 - loss: 0.0924
Epoch 15: val_accuracy did not improve from 0.99500
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9742 - loss: 0.0889 - val_accuracy: 0.9947 - val_loss: 0.0160 - learning_rate: 0.0010
Epoch 16/30
936/938 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9744 - loss: 0.0880
Epoch 16: val_accuracy did not improve from 0.99500
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9747 - loss: 0.0867 - val_accuracy: 0.9947 - val_loss: 0.0163 - learning_rate: 0.0010
Epoch 17/30
936/938 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9751 - loss: 0.0859
Epoch 17: val_accuracy did not improve from 0.99500
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9758 - loss: 0.0839 - val_accuracy: 0.9947 - val_loss: 0.0175 - learning_rate: 0.0010
Epoch 18/30
933/938

938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9773 - loss: 0.0774 - val_accuracy: 0.9952 - val_loss: 0.0153 - learning_rate: 5.0000e-04
Epoch 21/30
930/938 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9801 - loss: 0.0692
Epoch 21: val_accuracy improved from 0.99520 to 0.99560, saving model to best_model_augmented.h5


938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9796 - loss: 0.0703 - val_accuracy: 0.9956 - val_loss: 0.0141 - learning_rate: 5.0000e-04
Epoch 22/30
930/938 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9802 - loss: 0.0680
Epoch 22: val_accuracy did not improve from 0.99560
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9807 - loss: 0.0680 - val_accuracy: 0.9956 - val_loss: 0.0144 - learning_rate: 5.0000e-04
Epoch 23/30
933/938 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9825 - loss: 0.0627
Epoch 23: val_accuracy did not improve from 0.99560
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9814 - loss: 0.0657 - val_accuracy: 0.9956 - val_loss: 0.0143 - learning_rate: 5.0000e-04
Epoch 24/30
930/938 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9817 - loss: 0.0629
Epoch 24: val_accuracy did not improve from 0.99560
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9807 - loss: 0.0669 - val_accuracy: 0.9955 - val_loss: 0.0135 - learning_rate: 5.0000e-04
Epo

In [ ]:
# ---------------
# BƯỚC 5 (MỚI): GRADIO VỚI MODEL MỚI VÀ LOGIC TỐT NHẤT
# ---------------
import gradio as gr
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image, ImageOps, ImageFilter # Đảm bảo import ImageFilter

# 1. Tải model MỚI NHẤT
try:
    best_model = load_model("best_model_augmented.h5") # Tải model MỚI
    print("Model 'best_model_augmented.h5' đã được tải thành công!")
except Exception as e:
    print(f"LỖI: Không thể tải 'best_model_augmented.h5'.")
    best_model = model

# ================================================================
# 2. ĐỊNH NGHĨA HÀM XỬ LÝ (Sử dụng lại logic 'Erosion')
# Model mới (Augmented) + Erosion (Làm mỏng) là sự kết hợp mạnh nhất
def predict_digit(img_dict):
    
    if img_dict is None:
        return {str(i): 0.0 for i in range(10)}
    if not isinstance(img_dict, dict):
        return {str(i): 0.0 for i in range(10)}
    if 'composite' not in img_dict or img_dict['composite'] is None:
        return {str(i): 0.0 for i in range(10)}
        
    image_data = img_dict['composite']

    try:
        pil_image = Image.fromarray(image_data).convert('L')
    except Exception as e:
        return {str(i): 0.0 for i in range(10)}
    
    # 2. Đảo ngược ảnh (thành trắng/nền_đen)
    img_inverted = ImageOps.invert(pil_image)
    
    # 3. LÀM MỎNG NÉT VẼ (EROSION)
    img_eroded = img_inverted.filter(ImageFilter.MinFilter(3))

    # 4. Tìm bounding box (trên ảnh đã làm mỏng)
    bbox = img_eroded.getbbox()
    if bbox is None:
        return {str(i): 0.0 for i in range(10)}
        
    # 5. Cắt (crop)
    img_cropped = img_eroded.crop(bbox)
    
    # 6. Tạo canvas đen và thêm padding (căn giữa)
    width, height = img_cropped.size
    new_size = max(width, height) + 40
    new_canvas = Image.new('L', (new_size, new_size), 0)
    paste_x = (new_size - width) // 2
    paste_y = (new_size - height) // 2
    new_canvas.paste(img_cropped, (paste_x, paste_y))
    
    # 7. Resize về 28x28
    img_resized = new_canvas.resize((28, 28), Image.Resampling.LANCZOS)
    
    # 8. Chuyển sang NumPy
    img_array = np.array(img_resized)
    
    # 9. Chuẩn hóa
    img_normalized = img_array.astype('float32') / 255.0
    
    # 10. Reshape
    img_reshaped = img_normalized.reshape(1, 28, 28, 1)
    
    # 11. Dự đoán
    probabilities = best_model.predict(img_reshaped)
    
    # 12. Định dạng Output
    confidences = {str(i): float(prob) for i, prob in enumerate(probabilities[0])}
    
    return confidences
# ================================================================

# 3. Tạo Giao diện (với nút Submit)
iface = gr.Interface(
    fn=predict_digit,
    inputs=gr.Sketchpad(height=280, width=280),
    outputs=gr.Label(num_top_classes=3, label="Dự đoán"),
    title="Nhận dạng Chữ số Viết tay (Model 3.0 - Augmented)",
    description="Vẽ một chữ số (từ 0 đến 9) vào ô bên trái. Nhấn 'Submit' để xem kết quả, hoặc 'Clear' để vẽ lại.",
)

# 4. Chạy giao diện
iface.launch(debug=True)

Model 'best_model_augmented.h5' đã được tải thành công!
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Keyboard interruption in main thread... closing server.


In [60]:
# ---------------
# BƯỚC 1: CÀI ĐẶT OPENCV VÀ MEDIAPIPE
# ---------------
!pip install opencv-python mediapipe
print("Cài đặt OpenCV và MediaPipe thành công!")

ERROR: Could not find a version that satisfies the requirement mediapipe (from versions: none)
ERROR: No matching distribution found for mediapipe
Cài đặt OpenCV và MediaPipe thành công!
